In [1]:
import os
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd 

import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import matplotlib.pyplot as plt

import sys 
from pathlib import Path
import plotly.express as px

module_path = ""
# file=os.path.join(module_path,"utils.py")
sys.path.append(module_path)
import utils as utils

In [2]:
pipeline_tasks= {
    ('Project1','Task1_1') : 
        {'start_date':'1 JAN 2022','end_date':'31 JAN 2022',
         'skillset':'AI_architect','uf':0.5} ,
    
    ('Project1','Task2_1') : 
        {'start_date':'1 FEB 2022','end_date':'28 FEB 2022',
         'skillset':'BI','uf':1},
    
    ('Project2','Task1_2') : 
        {'start_date':'1 MAR 2022','end_date':'30 MAR 2022',
         'skillset':'DS','uf':0.5} ,  # --- > For Kareem
    
    ('Project2','Task2_2') : 
        {'start_date':'1 MAR 2022','end_date':'31 MAR 2022',
         'skillset':'DS','uf':1} ,
    
    ('Project2','Task3_2') : 
        {'start_date':'1 MAR 2022','end_date':'30 MAR 2022',
         'skillset':'DS','uf':0.5} ,
    ('Project3','Task1_3') : 
        {'start_date':'5 MAR 2023','end_date':'7 MAR 2023',
         'skillset':'BI','uf':1}  ,
    
     ('Project3','Task2_3') : 
        {'start_date':'1 JAN 2022','end_date':'5 JAN 2022',
         'skillset':'DS','uf':0.5} , # --- > For Kareem

}


RESOURCES= {
    'Amr' : 
        {'skillset':'AI_architect','seniority': 'LEAD 2C','slots':1,
        'current_load':{
            'ong_1':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':0.5} ,
            'ong_2':{'project':'MEWA','start_date':'1 JAN 2022','end_date':'31 MAR 2022' ,'uf':0.5}
        }},
    
    'Reem' : 
        {'skillset':['BI','VIS'],"avail_start":'1 FEB 2022',"avail_end": '28 FEB 2022','slots':1,
        'current_load':{
            'ong_3':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
        }},
    
    'Kareem' : 
        {'skillset':'DS',"avail_start":'2 FEB 2022',"avail_end": '30 MAR 2023','slots':1,
        'current_load':{
            'ong_4':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
        }},
    
    'Mohamed' : 
        {'skillset':'BI',"avail_start":'1 MAR 2022',"avail_end": '30 MAR 2022','slots':1,
        'current_load':{
            'ong_5':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
        }},
    
    'other1' :
        {'skillset':'Devops',"avail_start":'1 JAN 2022',"avail_end": '31 JAN 2022','slots':1,
        'current_load':{
            'ong_6':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
        }}
 
}

In [3]:
pd_1=utils.convert_to_pd(pipeline_tasks)
utils.plot_gantt_chart(pd_1)

In [4]:
ongoing_tasks=utils.resource_tasks(RESOURCES)
pipeline_tasks=utils.add_pipeline_type(pipeline_tasks)
TASKS=utils.merge_ongoing_pipeline(pipeline_tasks,ongoing_tasks)

all_tasks_overlap_tuples=utils.get_over_lap_tasks(TASKS)
all_task_overlap=utils.task_overlap_combination(all_tasks_overlap_tuples)

In [5]:
all_task_overlap

[[('Project1', 'Task1_1'), ('SWCC', 'SWCC_0')],
 [('Project1', 'Task1_1'), ('MEWA', 'MEWA_1')],
 [('SWCC', 'SWCC_0'), ('MEWA', 'MEWA_1')],
 [('Project1', 'Task2_1'), ('SWCC', 'SWCC_4')],
 [('Project2', 'Task1_2'), ('Project2', 'Task2_2')],
 [('Project2', 'Task1_2'), ('Project2', 'Task3_2')],
 [('Project2', 'Task1_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task2_2'), ('Project2', 'Task3_2')],
 [('Project2', 'Task2_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
 [('SWCC', 'SWCC_3'), ('Project3', 'Task2_3')],
 [('Project1', 'Task1_1'), ('SWCC', 'SWCC_0'), ('MEWA', 'MEWA_1')],
 [('Project2', 'Task1_2'), ('Project2', 'Task2_2'), ('Project2', 'Task3_2')],
 [('Project2', 'Task1_2'), ('Project2', 'Task2_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task1_2'), ('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task2_2'), ('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task1_2'),
  ('Project2', 'Task2_2'),
  ('Project2', 'Task3_2'),
  ('SWCC', 'SWCC

In [8]:
pipline_constraints,resource_constraints=utils.constraints_uf(all_task_overlap,TASKS)

In [9]:
new_resource_constraint=utils.reformulate_resource_constraints(resource_constraints,TASKS)

In [10]:
pipline_constraints

[[('Project2', 'Task1_2'), ('Project2', 'Task2_2')],
 [('Project2', 'Task1_2'), ('Project2', 'Task3_2')],
 [('Project2', 'Task2_2'), ('Project2', 'Task3_2')],
 [('Project2', 'Task1_2'), ('Project2', 'Task2_2'), ('Project2', 'Task3_2')]]

In [11]:
pipeline_project_tasks={}
for p,t in pipeline_tasks.keys():
    list_1=pipeline_project_tasks.get(p,[])
    list_1.append(t)
    pipeline_project_tasks[p]=list_1
print(pipeline_project_tasks)
print("\n")

pipeline_task_projects={}
for p,t in pipeline_tasks.keys():
    list_1=pipeline_project_tasks.get(t,[])
    list_1.append(p)
    pipeline_task_projects[t]=list_1
print(pipeline_task_projects)

{'Project1': ['Task1_1', 'Task2_1'], 'Project2': ['Task1_2', 'Task2_2', 'Task3_2'], 'Project3': ['Task1_3', 'Task2_3']}


{'Task1_1': ['Project1'], 'Task2_1': ['Project1'], 'Task1_2': ['Project2'], 'Task2_2': ['Project2'], 'Task3_2': ['Project2'], 'Task1_3': ['Project3'], 'Task2_3': ['Project3']}


In [12]:
model = ConcreteModel()
model.pipeline_tasks = Set(initialize = pipeline_tasks.keys(), dimen=2)
model.pipeline_taskids_only=Set(initialize=list(map(lambda x : x[1],pipeline_tasks.keys())),dimen=1)

model.pipeline_projects = Set(initialize = list(set([j for (j,m) in model.pipeline_tasks])))

model.RESOURCES =  Set(initialize = RESOURCES.keys(), dimen=1)

model.Resource_assignment=Set(initialize=model.pipeline_tasks * model.RESOURCES, dimen=3)

# model.Resource_assignment_proper=     Set(
#         initialize=model.pipeline_tasks * model.RESOURCES, dimen=3 ,
#        filter = lambda model,p,t,r : TASKS[(p,t)]['skillset'] in RESOURCES[r]['skillset']
#                 )
print("\n")
print(model.Resource_assignment.value_list)



    .ordered_data() to retrieve the values from a finite set in a
    deterministic order.  (deprecated in 5.7) (called from
    C:\jupyter\Anaconda3\envs\mo\lib\site-
    packages\IPython\core\interactiveshell.py:3398)
[('Project1', 'Task1_1', 'Amr'), ('Project1', 'Task1_1', 'Reem'), ('Project1', 'Task1_1', 'Kareem'), ('Project1', 'Task1_1', 'Mohamed'), ('Project1', 'Task1_1', 'other1'), ('Project1', 'Task2_1', 'Amr'), ('Project1', 'Task2_1', 'Reem'), ('Project1', 'Task2_1', 'Kareem'), ('Project1', 'Task2_1', 'Mohamed'), ('Project1', 'Task2_1', 'other1'), ('Project2', 'Task1_2', 'Amr'), ('Project2', 'Task1_2', 'Reem'), ('Project2', 'Task1_2', 'Kareem'), ('Project2', 'Task1_2', 'Mohamed'), ('Project2', 'Task1_2', 'other1'), ('Project2', 'Task2_2', 'Amr'), ('Project2', 'Task2_2', 'Reem'), ('Project2', 'Task2_2', 'Kareem'), ('Project2', 'Task2_2', 'Mohamed'), ('Project2', 'Task2_2', 'other1'), ('Project2', 'Task3_2', 'Amr'), ('Project2', 'Task3_2', 'Reem'), ('Project2', 'Task3_2', 'Kar

In [13]:
model.prt_improper_var=Var(model.Resource_assignment,within=Binary, initialize=0)

model.ra_var=Var(model.RESOURCES,within=NonNegativeReals, initialize=0)


model.ta_var=Var(model.pipeline_taskids_only,within=Binary, initialize=0)

In [14]:
pipeline_project_tasks

{'Project1': ['Task1_1', 'Task2_1'],
 'Project2': ['Task1_2', 'Task2_2', 'Task3_2'],
 'Project3': ['Task1_3', 'Task2_3']}

In [15]:
#CONSTRAINT FOR MAPPING TWO vars together (resources & Resource assignment)

model.cons = ConstraintList()
for resource in model.RESOURCES :
#     print(resource)
    model.cons.add(
    sum(model.prt_improper_var[p,t,resource] for p in model.pipeline_projects for t in pipeline_project_tasks[p]  ) ==
    model.ra_var[resource] 
    )

for task in model.pipeline_taskids_only:
    model.cons.add(
    sum(model.prt_improper_var[p,task,r] for p in pipeline_task_projects[task] for r in model.RESOURCES  ) ==
    model.ta_var[task] 
    )

In [16]:
#Any given task should have one or zero assignment
for t in model.pipeline_taskids_only:  
        model.cons.add(
                1 >= sum(  model.prt_improper_var[p,t,r] 
                         for r in model.RESOURCES 
                         for p in pipeline_task_projects[t])
            )

In [17]:
resource_constraints['Kareem']

[[('Project2', 'Task1_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task2_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
 [('SWCC', 'SWCC_3'), ('Project3', 'Task2_3')],
 [('Project2', 'Task1_2'), ('Project2', 'Task2_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task1_2'), ('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task2_2'), ('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
 [('Project2', 'Task1_2'),
  ('Project2', 'Task2_2'),
  ('Project2', 'Task3_2'),
  ('SWCC', 'SWCC_3')]]

In [18]:
resource_constraints

{'Amr': [[('Project1', 'Task1_1'), ('SWCC', 'SWCC_0')],
  [('Project1', 'Task1_1'), ('MEWA', 'MEWA_1')],
  [('Project1', 'Task1_1'), ('SWCC', 'SWCC_0'), ('MEWA', 'MEWA_1')]],
 'Mohamed': [[('Project1', 'Task2_1'), ('SWCC', 'SWCC_4')]],
 'Kareem': [[('Project2', 'Task1_2'), ('SWCC', 'SWCC_3')],
  [('Project2', 'Task2_2'), ('SWCC', 'SWCC_3')],
  [('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
  [('SWCC', 'SWCC_3'), ('Project3', 'Task2_3')],
  [('Project2', 'Task1_2'), ('Project2', 'Task2_2'), ('SWCC', 'SWCC_3')],
  [('Project2', 'Task1_2'), ('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
  [('Project2', 'Task2_2'), ('Project2', 'Task3_2'), ('SWCC', 'SWCC_3')],
  [('Project2', 'Task1_2'),
   ('Project2', 'Task2_2'),
   ('Project2', 'Task3_2'),
   ('SWCC', 'SWCC_3')]]}

In [19]:
new_resource_constraint

{'Amr': [{'ongoing': [('SWCC', 'SWCC_0')],
   'pipeline': [('Project1', 'Task1_1')]},
  {'ongoing': [('MEWA', 'MEWA_1')], 'pipeline': [('Project1', 'Task1_1')]},
  {'ongoing': [('SWCC', 'SWCC_0'), ('MEWA', 'MEWA_1')],
   'pipeline': [('Project1', 'Task1_1')]}],
 'Mohamed': [{'ongoing': [('SWCC', 'SWCC_4')],
   'pipeline': [('Project1', 'Task2_1')]}],
 'Kareem': [{'ongoing': [('SWCC', 'SWCC_3')],
   'pipeline': [('Project2', 'Task1_2')]},
  {'ongoing': [('SWCC', 'SWCC_3')], 'pipeline': [('Project2', 'Task2_2')]},
  {'ongoing': [('SWCC', 'SWCC_3')], 'pipeline': [('Project2', 'Task3_2')]},
  {'ongoing': [('SWCC', 'SWCC_3')], 'pipeline': [('Project3', 'Task2_3')]},
  {'ongoing': [('SWCC', 'SWCC_3')],
   'pipeline': [('Project2', 'Task1_2'), ('Project2', 'Task2_2')]},
  {'ongoing': [('SWCC', 'SWCC_3')],
   'pipeline': [('Project2', 'Task1_2'), ('Project2', 'Task3_2')]},
  {'ongoing': [('SWCC', 'SWCC_3')],
   'pipeline': [('Project2', 'Task2_2'), ('Project2', 'Task3_2')]},
  {'ongoing': [('S

In [20]:
pipeline_project_tasks

{'Project1': ['Task1_1', 'Task2_1'],
 'Project2': ['Task1_2', 'Task2_2', 'Task3_2'],
 'Project3': ['Task1_3', 'Task2_3']}

In [21]:
for p in model.pipeline_projects:
    for t in pipeline_project_tasks[p]:
        for r in model.RESOURCES :            
            if  TASKS[(p,t)]['skillset'] not in RESOURCES[r]['skillset']:
                model.cons.add(
                        0== model.prt_improper_var[p,t,r]
                        )

In [22]:
#For any overlap tasks the sum of utiliztion factor should be less than or equal 
for r in model.RESOURCES:
        

    # pipleine constraints (can apply to any resource of the specific skillset)       
    for pipline_constraint in pipline_constraints :        
        if RESOURCES[r]['skillset']==TASKS[pipline_constraint[0]]['skillset'] :           
            
            
            model.cons.add(
                 1 >= sum( TASKS[proj_task]['uf'] * model.prt_improper_var[proj_task[0],proj_task[1],r] 
                     for proj_task in pipline_constraint )
            )
            
            
    # resource constraints (specific to this resource)  
        
    for resource_constraint in new_resource_constraint.get(r,[]) : 
        
        model.cons.add(        
             1 >= sum( TASKS[proj_task]['uf'] * model.prt_improper_var[proj_task[0],proj_task[1],r] 
                     for proj_task in resource_constraint['pipeline'])
            
                  + sum(TASKS[proj_task]['uf'] for proj_task in resource_constraint['ongoing'])
                    )


In [23]:
expr=6 * sum(model.ra_var[r] for r in model.RESOURCES) + 4 * sum(model.ta_var[t] for t in model.pipeline_taskids_only)
model.objective=Objective(expr=expr,sense=maximize)

In [24]:
os.environ['NEOS_EMAIL'] = 'amr.mansour@devoteam.com'
opt = SolverFactory('cbc')  # Select solver
solver_manager = SolverManagerFactory('neos')  # Solve in neos server
results = solver_manager.solve(model, opt=opt)

In [25]:
def get_resource_assignment(resource_assignment):
    resource_table = {
        p: {t: [] for t in pipeline_project_tasks[p]} 
        for p in model.pipeline_projects}
    
    
    for p in model.pipeline_projects:
        for t in pipeline_project_tasks[p]:
            for r in model.RESOURCES :
                    if model.prt_improper_var[p, t, r].value == 1:
                        resource_table[p][t].append(r)
    return resource_table

get_resource_assignment(model.prt_improper_var)

{'Project2': {'Task1_2': [], 'Task2_2': [], 'Task3_2': []},
 'Project3': {'Task1_3': ['Mohamed'], 'Task2_3': []},
 'Project1': {'Task1_1': [], 'Task2_1': ['Reem']}}